In [1]:
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import torch

import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models

%matplotlib inline

In [2]:
data = np.load("./models/register1_images.npy", allow_pickle=True)

In [7]:
class AttendanceDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])
    
    def __getitem__(self, index):
        x = self.data[index]
        return self.transform(x)
    
    def __len__(self):
        return len(self.data)

myDataset = AttendanceDataset(data)
myLoader = torch.utils.data.DataLoader(myDataset, batch_size=64, shuffle=True)

In [8]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        # first conv layer, downsampling layer, and normalization layer
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=3, stride=2, padding=1, dilation=1),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=3, stride=2, padding=1, dilation=1),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d( kernel_size=3, stride=2, padding=1)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=2, padding=1, dilation=1),
            nn.ReLU()
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=2, dilation=1),
            nn.ReLU()
        )
        
        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=2, dilation=1),
            nn.ReLU(),
            nn.MaxPool2d( kernel_size=3, stride=2)
        )
        
        self.fc_layer_1 = nn.Sequential(
            nn.Linear(in_features=(256 * 2 * 2), out_features=1024),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=True),
        )
        
        self.fc_layer_2 = nn.Sequential(
            nn.Linear(in_features=1024, out_features=512),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=True)
        )
        
        self.output_layer = nn.Sequential(
            nn.Linear(in_features=512, out_features=128),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(-1, 256 * 2 * 2)  # reduce the dimensions for linear layer input
        x = self.fc_layer_1(x)
        x = self.fc_layer_2(x)
        return self.output_layer(x)

In [9]:
model = Model()
model.cuda()
print(model)

Model(
  (layer1): Sequential(
    (0): Conv2d(1, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
  (layer5): Sequential(
    (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer_1): Sequ

In [10]:
# Load checkpoint uncomment code
checkpoint = torch.load('./models/emnsit_model.pth')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [ ]:
model.eval()

for images, labels in myLoader: